<a href="https://colab.research.google.com/github/leniwiecc/SQLAlchemy-klocek/blob/main/Klocek_SQLAlchemy_cz1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacja i konfiguracja sqlalchemy. 
Mam nadzieję że wszystko będzie jasne i czytelne. Jak coś nie gra to pisać śmiało. Nie gryzę :D
1.	Sprawdzamy poleceniem pip czy mamy zainstalowaną bibliotekę flask_sqlalchemy. (pip3 freeze)
2.	Jeżeli nie no instalujemy: pip3 install flask_sqlalchemy


In [ ]:
 pip3 install flask_sqlalchemy

Aby zacząć zabawę z FlaskAlchemy musimy ją dodać do naszej aplikacji. Wklejamy do naszego kodu nastpeujące linie:

In [ ]:
from flask_sqlalchemy import SQLAlchemy


app=Flask(__name__)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db_test.db'
 ## UWAGA: db_test można zamienić na własną nazwkę 

db = SQLAlchemy(app)



Teraz musimy stworzyć naszą pierszą tabelę:

In [ ]:
class USERS(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user= db.Column(db.String(20), nullable=False, unique=True)
    password= db.Column(db.String(20), nullable=False)
    mail=db.Column(db.String(50), nullable=False, unique=True)
    city=db.Column(bd.String(200), nullable=True)
    date_jointed = db.Column(db.Date, default = datetime.date.today())
    

Kolumna id: db.column(db.Integer) <- tak definiujemy pole (kolumnę) i przyjmujemy że będą w niej tylko liczby całkowite. Przyjęło się że pole id to numer indetyfikacyjny dla danego wpisu. Taki 'pesel'. Jednoznacznie określa dany, konretny wpis. Aby system wiedział, że to jest to specjalne pole dodajmy komendę: primary_key=True. 

Kolumna user. Definiujmey kolumnę, ustawiamy że przechowywany tu będzie tekst o długości do 20 znaków. Możemy to zmienić na 5 , 24, 254. Komenda nullable=False informuje system że ta wartość musi zostać określona. Nie można dodać wpisu bez wartości usera. Metoda unique=True informuje nas że tylko 1 wpis może posiadać taką nazwę. Jeżeli sie gdzieś próbowaliście zarejstrować być może zdarzyło wam się dostać komunikat: taki użytkownik już istnieje. Podaj inną nazwę. 

Kolumna city: jak wyżej. Definiujemy skąd pochodzi dany użytkownik. Dodając metodę nullable=True uznajemy że nie każdy użytkownik chce podawać miasto z którgo pochodzi. Nie każdy chce się dzielić informacją że pochodzi z Sosnowca :D

Kolumna date_jointed: tutaj definiujemy typ pola jako data. Logiczne: data dołączenia i przyjmuje parametr domyślny (czyli jak dodajemy kolejny wpis to nie musimy dodawać pola data_dolaczenia w formularzu. Ona się automatycznie  będzie dodawać. datetime.date.today() to czysty kod pythona. Można to inaczej napisać, można to pominąć i dodać odpowiednią funkcję w chwili dodawania wpisu do bazy. Tutaj mamy dowolność. 

Czy tylko takie dane możemy wstawiać bo bazy? Ależ nie. Typów danych jest więcej, to jest tylko wstęp do tematu. Jak ktoś ma ochotę to zapraszam do dokumentacji: https://docs.sqlalchemy.org/en/14/core/type_basics.html#generic-types

No dobra, mamy już napisany szkielet tabeli to teraz należy ją zainicjalizować. Możemy to zrobić 'w locie' czyli w konsoli albo w osobnym pliku pythonowskim. Żeby już nie kombinować wszystko poniżej będziemy robić w konsoli. (to odpalmy konsolę/ terminal w miejscu w którym mamy naszą aplikację app.py)

In [ ]:
from app import db ## z naszej aplikacji pobieram db. 
from app import  USERS ## to jest nasza klasa którą storzylismy chwilę temu.
db.create_all() ## polecenie które tworzy tabelę w bazie danych .

Jeżeli nie otrzymaliśmy żandego komunikatu błędu to brawo Wy. Mamy swoją pierwszą tabelę. Zapełnijmy ją przykładowymi danymi:

In [ ]:
nowy_user= USERS(user='rafal', password='rafalto@1', mail='rafal@o2.pl', city='sosnowiec' ) ## definicja nowego użytkownika
db.session.add(nowy_user) ## dodajemy wpis do bazy
db.session.commit() ## zapisujemy nowy rekord w bazie. 


Proste? 3 linie kodu i wszystko gra. Dane o nowym użytkowniku teraz zostały 
opisane tu na sztywno. Pamiętajmy że będą one zwykle pochodziły z formularzy 
i tam będą weryfikowane, więc nie zachodzi tutaj potrzeba ponowngo ich sprawdzania (np czy hasło nie jest za proste).

Żeby przejść dalej do zapytań należy mieć jakieś dane w bazie więc powtarzamy dodawanie użytkowników kilka razy. Proste ćwiczenie: skopiuj te 3 linie kodu za każdym razem wpsiując inne imię, hasło, mail. Miasto możemy pominąć. Jeżeli już mamy parę wpisów to możemy przejść dalej.

Najprostszym zapytaniem jest pobieranie wpisów z bazy:

In [ ]:
users= USERS.query.all()

for user in users:
  print(user.id, user.name, user.date_jointed)

Czyli pobieramy wszystkie wpisy z bazy (.all) i później je wypisujemy w pętli. Jeżeli chemy zobaczyć ile mamy wpisów w bazie to na końcu dopiszemy jeszcze: .count()

In [ ]:
users = USERS.query.count()

Jeżeli chcemy te dane jakoś posortować to robimy to metodą .order_br(nazwa_kolumny.desc/ asc) <- czyli sortujemy po danej kolumnie rosnąco lub malejąco



In [ ]:
users = USERS.query.order_by(USERS.date_jointed.asc()).all()

A teraz coś ciekawszego: filtrowanie. Na przykład chemy pobrać wszystkich użykowników pochodzących z np: Wrocławia

In [ ]:
users=USERS.query.filter_by(city='Wrocław').all() ## pobieranie wszystkich użytkowniów z  Wrocławia
users=USERS.query.filter_by(city='Wrocław').count() ## dostaniemy liczbę użytkowników z Wrocławia.

**UWAGA**: metoda .all() pobiera wszystie wyniki (spełniające kryteria filtrów) i zwraca **listę obiektów**. Jeżeli mamy 3 wyniki to zmienna users 3 obiektami (zachowujacym sie jak lista). I każdy z tych elementów też będzie obiektem. Nawet jak będzie to wynik 1 wierszowy to dostajemy obiekt. Ale możemy to lekko zmodyikować zmienając metodę .all() na .first()
**Dajcie znać czy to jest jasne, zrozumiałe**

**Aktualizacja wypisów**
Mamy już użytkowniów w bazie ale musimy nieco zmdyfikować dane. Np: użytkownik chce zmienic swój email. 

In [ ]:
user_to_update = USERS.query.filter_by(name = 'rafal').first() ## wiemy że dostaniemy tylko 1 wynik. Musimy mieć zdefiniowany konkretny wpis który chcemy zmodyfikowac.
user_to_update.mail = 'nowy@mail.com' ## przypisujemy nową wartość
db.session.commit() ## zatwierdzamy zmianny. I gotowe. 

**Usuwanie wpisów.**
Usuwanie wpisów jest dość podobne do uaktualniania

In [ ]:
user_to_delate= = USERS.query.filter_by(name = 'rafal').first() 
db.session.delete(user_to_delate)
db.session.commit() ## zatwierdzamy zmianny. I gotowe. 

**Zaawansowane filtrowanie**
Oczywiście że możemy filtrować nasze dane w bardziej zaawansowany sposób. 

In [ ]:
user = USER.db.query.filter(USER.city.like('%Góra%')).all() 

Powyższe zaspytanie szuka wszystkich użytkowników którzy mieszkają w mieście ze słowem Góra. Czyli: z Jelenia Góra, Biała Góra, Góra Mała itp... Możemy też dodać więcej kryteriów szukania:

In [ ]:
user = USER.db.query.filter(USER.city.like('%Góra%'), USER.name.like('Bar%')).all() 

Jak wyżej tylko teraz imię musi zaczynać się od znaków Bar. Np: Bartek, Bartosza, Bartłomiej, Barbara itp....
> Koniec części pierwszej. Dajcie znać czy wszystko jest jasne i czytelne. W następnej części będzie trochę teorii o relacjach i zbudujemy bardziej rozbudowaną bazę :)